**1. Use Spark and Databricks to run ELT processes by creating fact tables** | 
The fact table Python scripts should contain appropriate keys from the dimensions. In addition, the fact table scripts should appropriately generate the correct facts based on the diagrams provided in the first step. 

**2. Use Spark and Databricks to run ELT processes by creating dimension tables** | 
The dimension Python scripts should match the schema diagram. Dimensions should generate appropriate keys and should not contain facts. 

**3. Produce Spark code in Databricks using Jupyter Notebooks and Python scripts** | 
The transform scripts should at minimum adhere to the following: should write to delta; should use overwrite mode; **save as a table in delta**.

## BUSINESS OUTCOMES

1. Analyze how much time spend per ride, based on
   
   * [ ] date and time factors such as day of week and time of day 
   * which station is starting and / or ending station
   * age of the rider at time of the ride
   * whether the rider is a (paying) member or casual rider
   
 2. Analyze how much money is spend: 

  * per month, quarter, year 
  * per member, based on the age of the rider at account start

In [0]:
silver = "silver.riders"
gold = "dimRider"
#create = "CREATE TABLE %s (id INT, firs VARCHAR(40), last VARCHAR (40), birthday DATE, account_start DATE, account_end DATE, is_member BOOLEAN )" % (gold)
#spark.sql(create)
silverdf = spark.sql(f"SELECT * FROM {silver}")
silverdf
silverdf.show()
df = silverdf
df.write.mode("overwrite").save(f"/delta/{gold}")

+----+-----------+---------+----------+--------------------+-------------+-----------+---------+
 id| first| last| birthday| address|account_start|account_end|is_member|
+----+-----------+---------+----------+--------------------+-------------+-----------+---------+
1000| Diana| Clark|1989-02-13| 1200 Alyssa Squares| 2019-04-23| null| true|
1001| Jennifer| Smith|1976-08-10| 397 Diana Ferry| 2019-11-01| 2020-09-01| true|
1002| Karen| Smith|1998-08-10|644 Brittany Row ...| 2022-02-04| null| true|
1003| Bryan| Roberts|1999-03-29|996 Dickerson Tur...| 2019-08-26| null| false|
1004| Jesse|Middleton|1969-04-11|7009 Nathan Expre...| 2019-09-14| null| true|
1005| Christine|Rodriguez|1974-08-27|224 Washington Mi...| 2020-03-24| null| false|
1006| Alicia| Taylor|2004-01-30| 1137 Angela Locks| 2020-11-27| 2021-12-01| true|
1007| Benjamin|Fernandez|1988-01-11| 979 Phillips Ways| 2016-12-11| null| false|
1008| John| Crawford|1987-02-21| 7691 Evans Court| 2021-03-28| 2021-07-01| true|
1009| Victoria| Ritter|1981-02-07|9922 Jim Crest Ap...| 2020-06-12| 2021-11-01| true|
1010| Tracy| Austin|1996-04-07| 92973 Mary Ville| 2019-12-27| null| true|
1011| Jessica| Mcgee|1984-12-29|950 Grimes Burg A...| 2017-05-20| null| true|
1012| Heather| Fisher|1980-10-20|65532 Davis Sprin...| 2021-10-16| null| true|
1013| Timothy| Jones|1985-07-10| 7757 Johnston Roads| 2020-12-28| 2021-11-01| true|
1014| Jennifer| Martin|1989-12-04| 501 Arellano Land| 2017-11-24| null| true|
1015|Christopher| Silva|2001-07-25|3710 Rodriguez Gl...| 2017-07-10| null| true|
1016| Andrew| Jones|1991-12-13| 72226 Casey Square| 2022-02-02| null| true|
1017| William| Lawson|1981-04-17| 40395 Terrell Parks| 2019-03-11| null| true|
1018| Shelly| Briggs|1986-09-02| 3514 Leslie Vista| 2021-07-25| null| false|
1019| Tina| Garcia|1997-05-03|00348 Brandi Park...| 2021-07-10| 2022-01-01| false|
+----+-----------+---------+----------+--------------------+-------------+-----------+---------+
only showing top 20 rows

In [0]:
silver = "silver.stations"
gold = "dimStation"

#create = "CREATE TABLE %s (id VARCHAR(50), name VARCHAR(150), \
#                           latitude FLOAT, longitude FLOAT )" % (gold)

silverdf = spark.sql(f"SELECT * FROM {silver}")
silverdf.show()
df = silverdf
df.write.mode("overwrite").save(f"/delta/{gold}")


+------------+--------------------+---------+----------+
 id| name| latitude| longitude|
+------------+--------------------+---------+----------+
 525|Glenwood Ave & To...| 42.0127| -87.66606|
KA1503000012| Clark St & Lake St|41.885796| -87.6311|
 637|Wood St & Chicago...|41.895634|-87.672066|
 13216| State St & 33rd St|41.834732|-87.625824|
 18003|Fairbanks St & Su...| 41.89581|-87.620255|
KP1705001026|LaSalle Dr & Huro...| 41.89488|-87.632324|
 13253|Lincoln Ave & Wav...|41.948795| -87.67528|
KA1503000044|Rush St & Hubbard St|41.890175| -87.62618|
KA1504000140|Winchester Ave & ...|41.924038|-87.676414|
TA1305000032|Clinton St & Madi...| 41.88224| -87.64107|
TA1306000012| Wells St & Huron St|41.894753| -87.6344|
 13133|Damen Ave & Cortl...| 41.91598| -87.67734|
 SL-005|Indiana Ave & Roo...| 41.86789| -87.62304|
 13235|Southport Ave & W...| 41.94815| -87.66394|
TA1307000139| MLK Jr Dr & 29th St|41.842052| -87.617|
TA1305000009|Clark St & Ida B ...| 41.87593|-87.630585|
 13276|Stockton Dr & Wri...|41.931347|-87.638695|
TA1307000107|Sheridan Rd & Mon...| 41.96167| -87.65464|
 13193|Larrabee St & Web...| 41.92182| -87.64414|
KA1503000072|Wacker Dr & Washi...|41.883133| -87.63732|
+------------+--------------------+---------+----------+
only showing top 20 rows

In [0]:
from pyspark.sql.functions import explode, sequence, to_date
from dateutil.relativedelta import relativedelta
import pyspark.sql.functions as F

silver = "silver.trips"

(beginDate, endDate) = spark.sql(f"SELECT min (to_date(start_at)) as beginDate, add_months(max (to_date(ended_at)),12) as endDate FROM {silver}").first()
endDate = endDate + relativedelta(months=24)

spark.sql(f"select explode(sequence(to_timestamp('{beginDate}'), (to_timestamp('{endDate}')) , interval 1 hour)) as ts") \
    .createOrReplaceTempView('dates')

create = """
create or replace table dimDate
USING delta
LOCATION '/delta/dimDate'
as select
  ts,
  hour(ts) AS hour,
  dayofweek(ts) as dayofweek,
  dayofmonth(ts) as dayofmonth,
  weekofyear(ts) as weekofyear,
  month(ts) as month,
  quarter(ts) as quarter,
  year(ts) AS year
from
  dates
"""

spark.sql(create)
spark.sql("optimize dimDate zorder by (ts)")


Out[40]: DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,numOutputCubes:bigint,mergedNumCubes:bigint>,numBatches:bigint,totalConsideredFiles:bigint,totalFilesSkipped:bigint,preserveInsertionOrder:boolean>]

In [0]:
silver = "silver.payments"
gold = "factPayment"

#create = "CREATE TABLE %s (id INT, date DATE, amount FLOAT, rider INT)" % (gold)
#spark.sql(create)

silverdf = spark.sql(f"SELECT * FROM {silver}")
silverdf.show()
df = silverdf
df.write.mode("overwrite").save(f"/delta/{gold}")


+---+----------+------+--------+
 id| date|amount|rider_id|
+---+----------+------+--------+
 1|2019-05-01| 9.0| 1000|
 2|2019-06-01| 9.0| 1000|
 3|2019-07-01| 9.0| 1000|
 4|2019-08-01| 9.0| 1000|
 5|2019-09-01| 9.0| 1000|
 6|2019-10-01| 9.0| 1000|
 7|2019-11-01| 9.0| 1000|
 8|2019-12-01| 9.0| 1000|
 9|2020-01-01| 9.0| 1000|
 10|2020-02-01| 9.0| 1000|
 11|2020-03-01| 9.0| 1000|
 12|2020-04-01| 9.0| 1000|
 13|2020-05-01| 9.0| 1000|
 14|2020-06-01| 9.0| 1000|
 15|2020-07-01| 9.0| 1000|
 16|2020-08-01| 9.0| 1000|
 17|2020-09-01| 9.0| 1000|
 18|2020-10-01| 9.0| 1000|
 19|2020-11-01| 9.0| 1000|
 20|2020-12-01| 9.0| 1000|
+---+----------+------+--------+
only showing top 20 rows

In [0]:
import pyspark.sql.functions as F

silver = "silver.trips"
rider = "silver.riders"
gold = "factTrip"

joineddf = spark.sql(f"""SELECT t.id, t.start_at, t.ended_at, t.duration, t.start_station, t.dest_station, t.rideable_type, t.rider_id, 
                                CAST (datediff (year, r.birthday, t.start_at) AS INTEGER) as rider_age 
                         FROM {silver} as t
                         LEFT JOIN {rider} as r ON t.rider_id = r.id
                      """)
df = joineddf.withColumn("duration",(F.col("ended_at").cast("int") - F.col("start_at").cast("int")))
df
df.show(5, truncate=False)
df.write.mode("overwrite").save(f"/delta/{gold}")

+----------------+-------------------+-------------------+--------+-------------+------------+-------------+--------+---------+
id |start_at |ended_at |duration|start_station|dest_station|rideable_type|rider_id|rider_age|
+----------------+-------------------+-------------------+--------+-------------+------------+-------------+--------+---------+
89E7AA6C29227EFF|2021-02-12 16:14:56|2021-02-12 16:21:43|407 |525 |660 |classic_bike |71934 |37 |
89E7AA6C29227EFF|2021-02-12 16:14:56|2021-02-12 16:21:43|407 |525 |660 |classic_bike |71934 |37 |
0FEFDE2603568365|2021-02-14 17:52:38|2021-02-14 18:12:09|1171 |525 |16806 |classic_bike |47854 |38 |
0FEFDE2603568365|2021-02-14 17:52:38|2021-02-14 18:12:09|1171 |525 |16806 |classic_bike |47854 |38 |
E6159D746B2DBB91|2021-02-09 19:10:18|2021-02-09 19:19:10|532 |KA1503000012 |TA1305000029|electric_bike|70870 |33 |
+----------------+-------------------+-------------------+--------+-------------+------------+-------------+--------+---------+
only showing top 5 rows